<a href="https://colab.research.google.com/github/SophieSchau/MRF_demo_ISMRM2022/blob/main/MRF_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MRF processing demo
This notebook will demonstrate the processing pipeline used in the abstract *Toward a 1-minute high-resolution brain exam - MR Fingerprinting with ML-synthesized contrasts and fast reconstruction* presented at the 2022 ISMRM Annual meeting in London (program number 53).

Before you start, Make sure you have a GPU enabled runtime (*Runtime->Change runtime type->set hardware accelerator to GPU*).

*This demo takes approximately 25 min to run(Reconstruction is 15-20min). So, please start running the cells before you start reading!*



## Overview
This project is aimed at translating highly undrsampled MRF to a clinically feasible tool, by building a robust reconstruction pipeline that is portable between different compute systems (research lab, hospital, high performance computing cluster, collaborators, etc...). To achieve this these core objectives were set:

- The pipeline should run smoothly on multiple systems.
- The pipeline should be easy to upgrade when the sequence, the reconstruction method, or the synthesis method is changed.
- The pipeline should be able to provide an image to send back to the scanner within 5 min.
- The pipeline should be able to send a series of images to PACS within ~30min.
- The pipeline should run on hardware available in clinical settings (for now, this means an 11GB GPU).

This modular MRF processing pipeline includes 4 steps:

1.   Read raw scan data and metadata(In the demo this step is replaced with downloading a demo dataset)
2.   Reconstruct and get coil compression from calibration scan
3.   Reconstruct MRF (fast subspace basis reconstruction)
4.   Synthesize clinical contrasts

Each step will be demonstrated here, and documentation on how to run the equivalent Docker containers on your own machine will be explained in a final section of this document.


## Data and code download 
*Run the next code block before reading this! The data takes around 5 minutes to download*

For this demo we have prepared a dataset consisting of five files:
1. `mrf_raw.npy` <- contains MRF kspace data
2. `calib_raw.npy` <- contains calibration GRE kspace data
3. `calib_nse.npy` <- contains noise measurement from the calibration GRE
4. `traj_grp16_inacc2.mat` <- contains the trajectory coordinates for the MRF
5. `subspace_basis_grp16.mat` <- contains the temporal basis components for the MRF


`mrf_raw.npy`, `calib_raw.npy`, and `calib_nse.npy` have been generated from ScanArchive files, which is the raw data output format used for MRI scanners by GE Healthcare. The conversion was done using containerized code provided by GE Healthcare. Since containers cannot easily be executed in Google Colab we skipped this step in the demo. However, if you want to run the conversion on your own system, you can run the containerized version on your own computer as explained in the final section of this demo.

*If your data hasn't downloaded yet, please sing happy birthday to yourself to pass the time, your birthday is soon/was recently anyway!*

In [ ]:
%%bash

mkdir tmp
cd tmp
wget -q -O data.zip -L https://stanfordmedicine.box.com/shared/static/61tx7yzaxrcw4ktrlpebwn1tom5urx6s
unzip -q data.zip
cd /content/
mv /content/tmp/MRF_Demo_ISMRM2022 /content/data
rm -rf tmp
mkdir /content/result
mkdir /content/result/synth/

git clone -q https://github.com/SophieSchau/MRF_demo_ISMRM2022.git
pip install -q git+https://github.com/mikgroup/sigpy.git@master

## Reconstruct and get coil compression from calibration scan
In this section we are going to use the raw k-space data from the calibration scan together with the noise measurment to perform coil whitening and [RoVir](https://doi.org/10.1002/mrm.28706) compression to remove any signal outside the central 256x256x256 mm field of view (where the MRF is to be reconstructed).

Each section of the pipeline is built up in its own folder in the repo, and has a `main.py` function that will perform the pipeline step. The idea is that if we change for example the calibration scan to a one with a different trajectory, all we need to do is update the data loader but the top level execution of the pipeline remains unchanged.

This example shows how we ran the calibration for the data shown at ISMRM 2022. The flags used here are:
- `--ksp` which sets the path to the raw calibration data
- `--nse` which sets the path to the noise measurement
-  `--ccm` which sets the location of the resulting coil compression matrix to use for the MRF acquisition
- `--rci` which we only include for demo purposes as it saves the rovir compressed calibration images so we can see the effect rovir has on the signal 
- `--nrc` sets the number of RoVir coils to compress down to (original number of coils in this case is 48)
- `--nsv` how many virtual coils to keep after svd compression (after RoVir).

You can also see all available flags and what they do by calling `main.py -h`.

Feel free to play around with different numbers of RoVir and SVD coils and see how that affects the signal in outside the center of the FOV. For example, not using RoVir at all (setting `--nrc` to 48) results in large signal originating in the neck outside the field-of-view.

In [ ]:
%%bash

cd /content/MRF_demo_ISMRM2022/src/01_calib/

python3 main.py --ksp /content/data/calib_raw.npy \
                --nse /content/data/calib_nse.npy \
                --ccm /content/result/ccm.npy \
                --rci /content/result/rovir_recon.npy \
                --nrc 40 --nsv 10 \

In [ ]:
# Let's have a look at the results of the calibration!
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

rovir_im = np.sum(np.abs(np.load('/content/result/rovir_recon.npy'))**2,axis=3) # sum of squares accross coils

for axis in [0,1,2]:
  fig, ax = plt.subplots()
  rect = patches.Rectangle((16, 16), 32, 32, linewidth=1, edgecolor='r', facecolor='none') # this rectangle defines the MRF field of view
  if axis is 0:
    ax.imshow(np.flipud(np.abs(rovir_im[:,:,32])), cmap='gray')
  elif axis is 1:
    ax.imshow(np.flipud(np.abs(rovir_im[:,32,:])), cmap='gray')
  elif axis is 2:
    ax.imshow(np.flipud(np.abs(rovir_im[32,:,:])), cmap='gray')
  ax.add_patch(rect)
  plt.axis('off')
  plt.show()


## MRF Reconstruction
*Again, we recommend running the next cell before reading through this as we have noticed that the reconstruction is much slower on Colab than when we run it locally (takes about 7 minutes locally and 15-20 min on Colab). Probably due to inefficient CPU to GPU data transfer.*

This is the main part of the reconstruction pipeline, where the MRF data is actually reconstructed into subspace basis maps.

Similarly, to the previous section , we move into the folder in the repository containing the `main.py` function for this part of the pipeline and then run it with a number of flags to define the reconstruction we want to run:
- `-c` reconstruction type (-c this means conjugate gradient, but we have also implemented -p, which is a [preconditioned locally-low-rank regularized reconstruction](https://submissions.mirasmart.com/ISMRM2022/itinerary/Files/PDFFiles/3483.html) that we used as ground truth in this project (this takes a long time to run so is omitted in this demo).
- `--phi` path to subspace basis
- `--trj` path to trajectory
- `--ksp` path to raw k-space data
- `--ccm` path to coil compression matrix (generated in previous section)
- `--res` path to where the resulting recosntruction should be saved
- `--yfv` run auto-FOV algorithm to center the image (this uses more RAM than Colab can manage so we will skip this step since the subject is inside the FOV already).
- `--mtx` matrix size
- `--ptt` number of points to remove from beginning of readout (this adds stability to the reconstruction)
- `--rnk` how many subspace coefficients to use
- `--mit` number of iterations
- `--nco` number of virtual coils to use

Again, you can see a full list of flags and changeable settings by running `main.py -h`.

In [ ]:
%%bash

cd /content/MRF_demo_ISMRM2022/src/02_recon/

python3 main.py -c --phi /content/data/subspace_basis_grp16.mat \
                   --trj /content/data/traj_grp16_inacc2.mat \
                   --ksp /content/data/mrf_raw.npy \
                   --ccm /content/result/ccm.npy \
                   --res /content/result/fast.npy \
                   --yfv False \
                   --mtx 256 --ptt 10 --rnk 3 --mit 40 --nco 3
                   

In [ ]:
fast = np.load('/content/result/fast.npy')

plt.rcParams['figure.figsize'] = [10, 30]

plt.subplot(1,3,1)
plt.imshow(np.rot90(np.flipud(np.abs(fast[:,:,150,0]))),cmap='gray', vmin=0, vmax=0.002)
plt.axis('off')
plt.subplot(1,3,2)
plt.imshow(np.rot90(np.flipud(np.abs(fast[:,:,150,1]))),cmap='gray', vmin=0, vmax=0.001)
plt.axis('off')
plt.subplot(1,3,3)
plt.imshow(np.rot90(np.flipud(np.abs(fast[:,:,150,2]))),cmap='gray', vmin=0, vmax=0.0004)
plt.axis('off')
plt.tight_layout()
plt.show()

## Contrast synthesis
*Did you finish the recon? Good job! Now it's just the easy part left, contrast synthesis! (this part takes about 3 min)*

For this part of the pipeline we again change folder and run the `main.py` function (you have probalbly gotten the gist now!). This function takes the recon from the previous step and uses it as input in a neural network that has been trained on 10 healthy volunteers to generate contrast weighted images.

The flags used in this step are:
- `--inp` the input data (subspace basis maps from the previous step)
- `--wgt` weights for the neural network. Note, the one we use here is for data with the settings for the recon above, if yu use different settings yu might get suboptimal results.
- `--sth` in which folder to save the synthesized images (folder must exist already)
- `--idx` indecies of contrasts to synthesize. They follow this mapping: 

  0. T1-weighted Cube
  1. T1-weighted MPRAGE
  2. T2-weighted Cube
  3. T2-weighted FLAIR Cube
  4. T2-weighted Double Inversion Recovery Cube

In [ ]:
%%bash
cd /content/MRF_demo_ISMRM2022/src/03_synth/

python3 main.py  --inp /content/result/fast.npy \
                 --wgt /content/MRF_demo_ISMRM2022/src/03_synth/checkpoints/v1_InitCG/800_net_G.pth \
                 --sth /content/result/synth \
                 --idx 0 1 2 3 4

In [ ]:
fast = np.load('/content/result/fast.npy')

for contrast in ['t1mprage', 't1w', 't2w', 't2flair', 'dir']:
  synth = np.load('/content/result/synth/' + contrast + '.npy')
  plt.rcParams['figure.figsize'] = [25, 15]

  plt.subplot(1,3,1)
  plt.imshow(np.rot90(np.flipud(np.abs(synth[:,:,150]))),cmap='gray')
  plt.axis('off')
  plt.subplot(1,3,2)
  plt.imshow(np.rot90(np.flipud(np.abs(synth[:,150,:]))),cmap='gray')
  plt.text(128,10,contrast, color='r', horizontalalignment='center', verticalalignment='top', fontsize=30)
  plt.axis('off')
  plt.subplot(1,3,3)
  plt.imshow(np.rot90(np.flipud(np.abs(synth[150,:,:]))),cmap='gray')
  plt.axis('off')
  plt.tight_layout()
  plt.show()

## Docker version for local execution

The aim of this project is real life application and Google Colab is not how we aim to deploy this work. If you want to try the real life version of it on a machine that has [Docker](https://www.docker.com) installed, as well as a 11GB or larger GPU, please follow these steps:


### Download data and code:
This dataset is smaller than the one used in the Colab demo, and consists of five files:
1. `ScanArchive_MRF.h5` <- contains MRF kspace data
2. `ScanArchive_GRE.h5` <- contains calibration GRE kspace data (as well as the noise mesurements)
3. `traj_grp16_inacc2.mat` <- contains the trajectory coordinates for the MRF (same as above)
4. `subspace_basis_grp16.mat` <- contains the temporal basis components for the MRF (same as above)
5. `python-sdk.tar` <- A Docker container provided by GE healthcare that can be used to read ScanArchive files.


```
git clone -q https://github.com/SophieSchau/MRF_demo_ISMRM2022.git
cd MRF_demo_ISMRM2022
mkdir data
cd data
wget -q -O data_docker.zip -L https://stanfordmedicine.box.com/shared/static/mvzz9lvpl8ixamesokypomyrnlx8pqs1
unzip -q data_docker.zip
cd ..
mv data/MRF_Demo_ISMRM2022_Docker/* data/.
mv data/python-sdk.tar src/00_io/python-sdk.tar
mkdir result
mkdir result/synth/
export WORKDIR=`pwd`

```
### Build the first Docker container:
Make sure you have Docker installed and running. Then run the following lines to build the data reader (Note, Docker containers can take a while to build, but you only need to build them once!):

```
cd ${WORKDIR}/src/00_io
docker load -i python-sdk.tar # This loads the GE provided container to read ScanArchive files.
docker build -t setsompop/scan_archive_io .
```

And run it with your local filesystem mounted so that it can read the ScanArchive files and write the numpy arrays that we will use for the rest of the pipeline.
```
docker run -v /:/mnt/:z setsompop/scan_archive_io \
             --scn /mnt/${WORKDIR}/data/ScanArchive_GRE.h5 \
             --ksp /mnt/${WORKDIR}/data/calib_raw.npy \
             --nse /mnt/${WORKDIR}/data/calib_nse.npy
```

We will use the same Docker container to read the MRF data (it can also be used to write Dicom images based on the metadata in the ScanArchive file, but we are skipping that for now!):

```
docker run -v /:/mnt/:z setsompop/scan_archive_io \
             --scn /mnt/${WORKDIR}/data/ScanArchive_MRF.h5 \
             --ksp /mnt/${WORKDIR}/data/mrf_raw.npy
```
### Matching the Colab tutorial
The rest of the steps will follow the Colab tutorial. Just remember to add `/mnt/` and `${WORKDIR}` to your path so that Docker can find and write the files to your local filesystem! Next code blocks shows you how to build and run each step in the pipe (same as you did in the Colab notebook).

#### Calibration:
```
cd ${WORKDIR}/src/01_calib
docker build -t setsompop/calib .
docker run --gpus all -v /:/mnt/:z setsompop/calib \
            --ksp /mnt/${WORKDIR}/data/calib_raw.npy \
            --nse /mnt/${WORKDIR}/data/calib_nse.npy \
            --ccm /mnt/${WORKDIR}/result/ccm.npy \
            --nrc 40 \
            --nsv 10 
```

#### MRF Reconstruction:
```
cd ${WORKDIR}/src/02_recon
docker build -t setsompop/recon .
docker run --gpus all -v /:/mnt/:z setsompop/recon \
            -c --phi /mnt/${WORKDIR}/data/subspace_basis_grp16.mat \
                   --trj /mnt/${WORKDIR}/data/traj_grp16_inacc2.mat \
                   --ksp /mnt/${WORKDIR}/data/mrf_raw.npy \
                   --ccm /mnt/${WORKDIR}/result/ccm.npy \
                   --res /mnt/${WORKDIR}/result/fast.npy \
                   --yfv False \
                   --mtx 256 --ptt 10 --rnk 3 --mit 40 --nco 3
```

#### MRF Synthesis:
```
cd ${WORKDIR}/src/03_synth
docker build -t setsompop/synth .
docker run --gpus all -v /:/mnt/:z setsompop/synth \
            --inp /mnt/${WORKDIR}/result/fast.npy \
                 --wgt /mnt/${WORKDIR}/src/03_synth/checkpoints/v1_InitCG/800_net_G.pth \
                 --sth /mnt/${WORKDIR}/result/synth \
                 --idx 0 1 2 3 4
```